# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev2"><a href="#1.1-Import-Data"><span class="toc-item-num">1.1 &nbsp;&nbsp;</span>Import Data</a></div>
 <div class="lev2"><a href="#1.2-Indexing-Data"><span class="toc-item-num">1.2 &nbsp;&nbsp;</span>Indexing Data</a></div>
 <div class="lev2"><a href="#1.3-Data-cleaning"><span class="toc-item-num">1.3 &nbsp;&nbsp;</span>Data cleaning</a></div>
 <div class="lev2"><a href="#1.4-Calculating-means"><span class="toc-item-num">1.4 &nbsp;&nbsp;</span>Calculating means</a></div>
 <div class="lev2"><a href="#1.5-Merging"><span class="toc-item-num">1.5 &nbsp;&nbsp;</span>Merging</a>
 </p>
 <p>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div>
 <div class="lev2"><a href="#Task-3.1"><span class="toc-item-num"></span>3.1 Types and Values</a></div>
 <div class="lev2"><a href="#Task-3.2"><span class="toc-item-num"></span>3.2 Histograms</a></div>
 <div class="lev2"><a href="#Task-3.3"><span class="toc-item-num"></span>3.3 Cabin Pie Chart</a></div>
 <div class="lev2"><a href="#Task-3.4"><span class="toc-item-num"></span>3.4 Class survival rate</a></div>
 <div class="lev2"><a href="#Task-3.5"><span class="toc-item-num"></span>3.5 Survival proportion per class per sex</a></div>
 <div class="lev2"><a href="#Task-3.6"><span class="toc-item-num"></span>3.5 Survival Data per age category, class and sex</a></div>
 </p>

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num">0.&nbsp;</span>Table of contents</a></div>
 


In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

In [ ]:
from datetime import date,time, datetime
import glob, os
from dateutil.parser import parse

In [ ]:
DATA_FOLDER = 'Data/' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average* per year of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

For each country:
    1. daily average
        1.1 new cases
            1.1.1 suspected
            1.1.2 probable
            1.1.3 confirmed 
        1.2 deaths

The idea was to calculate each mean and then sum the new value for each report over roughly a month and divide the result by the day-span between the first and last report. For example if the data begin the 15 June, we takes values until 15 July (roughly one month). 
However for December in Liberia, we had some problems with data between the beginning and the end of this month (there were less death at the end than at the beginning). Futhermore we have good values from June to November for the Liberia and that's why in the last Dataframe we have only 0 for this month for this country. 
For the Sierra Leone you can see that we have a "date_death_mean" from the previous month. This is because the values for the death are cumulative. So we needed to substract by the mean of the previous month. However the data seems corrupted too because they just change month after month. So if we took a date and exctract a value from a date it will be exactly the same. This is why we took only one day to do our mean of the previous month

## 1.1 Import Data
We want to import all the files for each country in a single Dataframe. We did this following a stackoverflow thread that allowed us to easily concatenate the files one by one

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
EBOLA_FOLDER = DATA_FOLDER + "ebola/"
GUINEA_FOLDER = EBOLA_FOLDER + "guinea_data/"
LIBERIA_FOLDER = EBOLA_FOLDER + "liberia_data/"
SL_FOLDER = EBOLA_FOLDER + "sl_data/"

Solution for following imports found at: https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe/21232849

### Guinea Import

In [ ]:
path = GUINEA_FOLDER
all_files = glob.glob(os.path.join(path, "*.csv"))

df_from_each_file = (pd.read_csv(f) for f in all_files)
guinea_data = pd.concat(df_from_each_file, ignore_index=True)

In [ ]:
guinea_data.head(2)

### Liberia Import

In [ ]:
path = LIBERIA_FOLDER
all_files = glob.glob(os.path.join(path, "*.csv"))

df_from_each_file = (pd.read_csv(f) for f in all_files)
liberia_data = pd.concat(df_from_each_file, ignore_index=True)

In [ ]:
liberia_data.head(2)

### SL Import

In [ ]:
path = SL_FOLDER # use your path
all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent

df_from_each_file = (pd.read_csv(f) for f in all_files)
sl_data = pd.concat(df_from_each_file, ignore_index=True)

In [ ]:
sl_data.head(2)

## 1.2 Indexing Data

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
guinea_data_indexed = guinea_data.set_index(['Description', 'Date'])
guinea_data_indexed.head(3)

In [ ]:
liberia_data['Date'] = pd.to_datetime(liberia_data['Date'])
#liberia_data.Date.apply(lambda d: datetime.strptime(d, '%m/%d/%y'))
liberia_data_indexed = liberia_data.set_index(['Variable', 'Date'])
liberia_data_indexed.head(3)

In [ ]:
sl_data['date'] = pd.to_datetime(sl_data['date'])
sl_data_indexed = sl_data.set_index(['variable', 'date'])
sl_data_indexed.head(3)

We observe a lot of NaN values and two duplicated columns in the Sierra Leone Data. This is the next step, cleaning

## 1.3 Data cleaning

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

First idea: it seems to make sense to replace NaN values by 0's. Then we can proceed easily to merging columns

In [ ]:
guinea_data_indexed = guinea_data_indexed.apply(pd.to_numeric, errors = 'coerce')
guinea_data_indexed.fillna(0, inplace=True)
guinea_data_indexed.head(3)

In [ ]:
liberia_data_indexed = liberia_data_indexed.apply(pd.to_numeric, errors = 'coerce')
liberia_data_indexed.fillna(0, inplace=True)
liberia_data_indexed.head(3)

In [ ]:
sl_data_indexed = sl_data_indexed.apply(pd.to_numeric, errors = 'coerce')
sl_data_indexed.fillna(0, inplace=True)
sl_data_indexed.head(3)

Now proceed to the merging of duplicated columns

In [ ]:
sl_merged = sl_data_indexed.copy()
sl_merged['Police training School'] = pd.concat([sl_data_indexed['Police training School'] +
                                                       sl_data_indexed['Police traning School']])
del sl_merged['Police traning School']
sl_merged['Western area'] = pd.concat([sl_data_indexed['Western area'] +
                                                       sl_data_indexed['Western area combined']])
del sl_merged['Western area combined']
sl_merged.head(3)

## 1.4 Calculating means

### Guinea Means

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
guinea_means = guinea_data_indexed['Totals']
guinea_means.head(3)

In [ ]:
guinea_means.sort_index(level=['Description','Date'], ascending=[1, 0], inplace=True)
guinea_filtered = guinea_means[['New cases of suspects', 'New cases of probables', 'New cases of confirmed', 
                               'New deaths registered']]
guinea_filtered = pd.DataFrame(guinea_filtered)
guinea_filtered.loc['New deaths registered']

In [ ]:
guinea_unstacked = guinea_filtered.unstack()
guinea_unstacked.index = ['New confirmed', 'New probables', 'New suspects', 'Death registered']
guinea_unstacked.fillna(0, inplace = True)
guinea_unstacked = guinea_unstacked['Totals']
guinea_unstacked

#### August Guinea Means

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
first_august_gui = '2014-08-04'
last_august_gui = '2014-09-04'
guinea_start_august = guinea_unstacked[[first_august_gui]]
guinea_end_august = guinea_unstacked[[last_august_gui]]

In [ ]:
august_month_gui = guinea_unstacked.loc[:,first_august_gui:last_august_gui] 
august_start = parse(guinea_start_august.columns[0])
august_end = parse(guinea_end_august.columns[0])

august_month_gui

In [ ]:
august_days = august_end - august_start
august_days = august_days.days

In [ ]:
guinea_august_means = august_month_gui.sum(axis = 1)/august_days
guinea_august_means

#### September Guinea Means

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
first_sept_gui = '2014-09-02'
last_sept_gui = '2014-10-01'
guinea_start_sept = guinea_unstacked[[first_sept_gui]]
guinea_end_sept = guinea_unstacked[[last_sept_gui]]

In [ ]:
sept_month_gui = guinea_unstacked.loc[:,first_sept_gui:last_sept_gui] 
sept_start_gui = parse(guinea_start_sept.columns[0])
sept_end_gui = parse(guinea_end_sept.columns[0])

sept_month_gui

In [ ]:
sept_days_gui = sept_end_gui - sept_start_gui
sept_days_gui = sept_days_gui.days

In [ ]:
guinea_sept_means = sept_month_gui.sum(axis = 1)/sept_days_gui
guinea_sept_means

### Liberia Means

In [ ]:
liberia_means = liberia_data_indexed['National']
#liberia_means = pd.DataFrame(liberia_means)
liberia_means.head(2)

In [ ]:
liberia_means.sort_index(level=['Variable', 'Date'], ascending=[1, 1], inplace=True)
liberia_filtered = liberia_means[['New Case/s (Suspected)', 'New Case/s (Probable)', 'New case/s (confirmed)', 
                               'Newly reported deaths']]
liberia_filtered.head(3)

In [ ]:
liberia_filtered = pd.DataFrame(liberia_filtered)
liberia_unstacked = liberia_filtered.unstack()
liberia_unstacked.index = ['New probable', 'New suspected', 'New confirmed', 'Death reported']
liberia_unstacked = liberia_unstacked['National']
liberia_unstacked

#### June Liberia Means

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
first_june_lib = date(2014, 6, 16)
last_june_lib = date(2014, 7, 17)
lib_start_june = liberia_unstacked[[first_june_lib]]
lib_end_june = liberia_unstacked[[last_june_lib]]

In [ ]:
june_month_lib = liberia_unstacked.loc[:,first_june_lib:last_june_lib] 
june_start_lib = lib_start_june.columns[0]
june_end_lib = lib_end_june.columns[0]

june_month_lib

In [ ]:
june_days_lib = june_end_lib - june_start_lib
june_days_lib = june_days_lib.days

In [ ]:
lib_june_means = june_month_lib.sum(axis = 1)/june_days_lib
lib_june_means

#### July Liberia Means

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
first_date_lib = date(2014, 7, 1)
last_date_lib = date(2014, 8, 2)
lib_start = liberia_unstacked[[first_date_lib]]
lib_end = liberia_unstacked[[last_date_lib]]

month_lib = liberia_unstacked.loc[:,first_date_lib:last_date_lib] 
month_start_lib = lib_start.columns[0]
month_end_lib = lib_end.columns[0]

days_lib = (month_end_lib - month_start_lib).days

lib_july_means = month_lib.sum(axis = 1)/days_lib
lib_july_means

#### August Liberia Means

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
first_date_lib = date(2014, 8, 2)
last_date_lib = date(2014, 9, 2)
lib_start = liberia_unstacked[[first_date_lib]]
lib_end = liberia_unstacked[[last_date_lib]]

month_lib = liberia_unstacked.loc[:,first_date_lib:last_date_lib] 
month_start_lib = lib_start.columns[0]
month_end_lib = lib_end.columns[0]

days_lib = (month_end_lib - month_start_lib).days

lib_aug_means = month_lib.sum(axis = 1)/days_lib
lib_aug_means

#### September Liberia Means

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
first_date_lib = date(2014, 9, 1)
last_date_lib = date(2014, 10, 1)
lib_start = liberia_unstacked[[first_date_lib]]
lib_end = liberia_unstacked[[last_date_lib]]

month_lib = liberia_unstacked.loc[:,first_date_lib:last_date_lib] 
month_start_lib = lib_start.columns[0]
month_end_lib = lib_end.columns[0]

days_lib = (month_end_lib - month_start_lib).days

lib_sep_means = month_lib.sum(axis = 1)/days_lib
lib_sep_means

#### October Liberia Means

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
first_date_lib = date(2014, 10, 1)
last_date_lib = date(2014, 10, 31)
lib_start = liberia_unstacked[[first_date_lib]]
lib_end = liberia_unstacked[[last_date_lib]]

month_lib = liberia_unstacked.loc[:,first_date_lib:last_date_lib] 
month_start_lib = lib_start.columns[0]
month_end_lib = lib_end.columns[0]

days_lib = (month_end_lib - month_start_lib).days

lib_oct_means = month_lib.sum(axis = 1)/days_lib
lib_oct_means

#### November Liberia Means

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
first_date_lib = date(2014, 11, 2)
last_date_lib = date(2014, 12, 2)
lib_start = liberia_unstacked[[first_date_lib]]
lib_end = liberia_unstacked[[last_date_lib]]

month_lib = liberia_unstacked.loc[:,first_date_lib:last_date_lib] 
month_start_lib = lib_start.columns[0]
month_end_lib = lib_end.columns[0]

days_lib = (month_end_lib - month_start_lib).days

lib_nov_means = month_lib.sum(axis = 1)/days_lib
lib_nov_means

#### December Liberia Means

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
# Less Death values at the beginning than at the end (corrupted data so we didn't take them)

### Sierra Leone Means

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
sl_means = sl_data_indexed['National']
sl_means.head(2)

In [ ]:
sl_means.sort_index(level=['variable','date'], ascending=[1, 1], inplace=True)
sl_filtered = sl_means[['new_noncase', 'new_suspected', 'new_probable', 'new_confirmed', 
                               'death_suspected', 'death_probable', 'death_confirmed']]
sl_filtered.head()

In [ ]:
sl_filtered = pd.DataFrame(sl_filtered)
sl_unstacked  = sl_filtered.unstack()

In [ ]:
sl_unstacked.reindex(['new_noncase', 'new_suspected', 'new_probable', 'new_confirmed', 
                               'death_suspected', 'death_probable', 'death_confirmed'])
sl_unstacked.index = ['Death confirmed', 'Death probable', 'Death suspected', 
                      'New confirmed', 'New noncase', 'New probable', 'New suspected'  
                     ]
sl_unstacked = sl_unstacked['National']
sl_unstacked.head(2)

#### August Sierra Leone Means

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
first_date_sl = date(2014, 8, 12)
last_date_sl = date(2014, 8, 31)
date_death_mean = date(2014, 8, 12)
sl_start = sl_unstacked[[first_date_sl]]
sl_end = sl_unstacked[[last_date_sl]]

month_sl = sl_unstacked.loc[:,first_date_sl:last_date_sl] 
month_start_sl = sl_start.columns[0]
month_end_sl = sl_end.columns[0]

days_sl = (month_end_sl - month_start_sl).days
death_c = (month_sl.loc['Death confirmed', last_date_sl] - month_sl.loc['Death confirmed', first_date_sl])/days_sl
sl_aug_means = month_sl.sum(axis = 1)/days_sl
sl_aug_means['Death confirmed'] = death_c
sl_aug_means

#### September Sierra Leone Means

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
first_date_sl = date(2014, 9, 1)
last_date_sl = date(2014, 9, 30)
date_death_mean = date(2014, 8, 15)
sl_start = sl_unstacked[[first_date_sl]]
sl_end = sl_unstacked[[last_date_sl]]

month_sl = sl_unstacked.loc[:,first_date_sl:last_date_sl] 
month_start_sl = sl_start.columns[0]
month_end_sl = sl_end.columns[0]

days_sl = (month_end_sl - month_start_sl).days
death_c = (month_sl.loc['Death confirmed', last_date_sl] - month_sl.loc['Death confirmed', first_date_sl])/days_sl
sl_sep_means = month_sl.sum(axis = 1)/days_sl
sl_sep_means['Death confirmed'] = death_c
sl_sep_means

#### October Sierra Leone Means

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
first_date_sl = date(2014, 10, 1)
last_date_sl = date(2014, 10, 31)
date_death_mean = date(2014, 9, 15)
sl_start = sl_unstacked[[first_date_sl]]
sl_end = sl_unstacked[[last_date_sl]]

month_sl = sl_unstacked.loc[:,first_date_sl:last_date_sl] 
month_start_sl = sl_start.columns[0]
month_end_sl = sl_end.columns[0]

days_sl = (month_end_sl - month_start_sl).days
death_c = (month_sl.loc['Death confirmed', last_date_sl] - month_sl.loc['Death confirmed', first_date_sl])/days_sl
sl_oct_means = month_sl.sum(axis = 1)/days_sl
sl_oct_means['Death confirmed'] = death_c
sl_oct_means['Death probable'] = (sl_oct_means['Death probable'] - sl_unstacked.loc['Death probable', date_death_mean])/days_sl
sl_oct_means['Death suspected'] = (sl_oct_means['Death suspected'] - 
                                   sl_unstacked.loc['Death suspected', date_death_mean])/days_sl
sl_oct_means

#### November Sierra Leone Means

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
first_date_sl = date(2014, 11, 1)
last_date_sl = date(2014, 11, 29)
date_death_mean = date(2014, 10, 4)
sl_start = sl_unstacked[[first_date_sl]]
sl_end = sl_unstacked[[last_date_sl]]

month_sl = sl_unstacked.loc[:,first_date_sl:last_date_sl] 
month_start_sl = sl_start.columns[0]
month_end_sl = sl_end.columns[0]

days_sl = (month_end_sl - month_start_sl).days
death_c = (month_sl.loc['Death confirmed', last_date_sl] - month_sl.loc['Death confirmed', first_date_sl])/days_sl
sl_nov_means = month_sl.sum(axis = 1)/days_sl
sl_nov_means['Death confirmed'] = death_c
sl_nov_means['Death probable'] = (sl_nov_means['Death probable'] - sl_unstacked.loc['Death probable', date_death_mean])/days_sl
sl_nov_means['Death suspected'] = (sl_nov_means['Death suspected'] - 
                                   sl_unstacked.loc['Death suspected', date_death_mean])/days_sl
sl_nov_means

#### December Sierra Leone Means

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
first_date_sl = date(2014, 12, 1)
last_date_sl = date(2014, 12, 6)
date_death_mean = date(2014, 10, 10)
sl_start = sl_unstacked[[first_date_sl]]
sl_end = sl_unstacked[[last_date_sl]]

month_sl = sl_unstacked.loc[:,first_date_sl:last_date_sl] 
month_start_sl = sl_start.columns[0]
month_end_sl = sl_end.columns[0]

days_sl = (month_end_sl - month_start_sl).days
death_c = (month_sl.loc['Death confirmed', last_date_sl] - month_sl.loc['Death confirmed', first_date_sl])/days_sl
sl_dec_means = month_sl.sum(axis = 1)/days_sl
sl_dec_means['Death confirmed'] = death_c
sl_dec_means['Death probable'] = (sl_dec_means['Death probable'] - sl_unstacked.loc['Death probable', date_death_mean])/days_sl
sl_dec_means['Death suspected'] = (sl_dec_means['Death suspected'] - 
                                   sl_unstacked.loc['Death suspected', date_death_mean])/days_sl
sl_dec_means

## 1.5 Merging

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

This par consists of combining the different results in a single Dataframe

#### Guinea Merge

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
guinea_august_means = pd.DataFrame(guinea_august_means)
guinea_august_means.columns = ['Aug']
guinea_august_means

In [ ]:
guinea_sept_means = pd.DataFrame(guinea_sept_means)
guinea_sept_means.columns = ['Sep']
guinea_sept_means

In [ ]:
guinea_merged = pd.concat([guinea_august_means, guinea_sept_means], axis=1)
guinea_merged

#### Liberia Merge

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
lib_june_means = pd.DataFrame(lib_june_means)
lib_june_means.columns = ['Jun']

lib_july_means = pd.DataFrame(lib_july_means)
lib_july_means.columns = ['Jul']

lib_aug_means = pd.DataFrame(lib_aug_means)
lib_aug_means.columns = ['Aug']

lib_sep_means = pd.DataFrame(lib_sep_means)
lib_sep_means.columns = ['Sep']

lib_oct_means = pd.DataFrame(lib_oct_means)
lib_oct_means.columns = ['Oct']

lib_nov_means = pd.DataFrame(lib_nov_means)
lib_nov_means.columns = ['Nov']

In [ ]:
lib_merged = pd.concat([lib_june_means, lib_july_means, lib_aug_means, lib_sep_means, lib_oct_means, lib_nov_means], axis=1)
lib_merged

#### Sierra Leone Merge

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
sl_aug_means = pd.DataFrame(sl_aug_means)
sl_aug_means.columns = ['Aug']

sl_sep_means = pd.DataFrame(sl_sep_means)
sl_sep_means.columns = ['Sep']

sl_oct_means = pd.DataFrame(sl_oct_means)
sl_oct_means.columns = ['Oct']

sl_nov_means = pd.DataFrame(sl_nov_means)
sl_nov_means.columns = ['Nov']

sl_dec_means = pd.DataFrame(sl_dec_means)
sl_dec_means.columns = ['Dec']

In [ ]:
sl_merged = pd.concat([sl_aug_means, sl_sep_means,  sl_oct_means, sl_nov_means, sl_dec_means], axis=1)
sl_merged

#### Countries Merge

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
merged = pd.concat([guinea_merged, lib_merged, sl_merged], keys = ['Guinea', 'Liberia', 'Sierra Leone']).fillna(0).round(1)
merged = merged.reindex_axis(['Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], axis = 1)
merged.index.names = ['Country', None]
merged

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
# Write your answer here

    
path ='Data/microbiome'
microbiome = pd.DataFrame()
new_index = pd.DataFrame()

for idx in range(1,10):
    data_mid = pd.read_excel(path + '/MID' + str(idx) + '.xls', 'Sheet 1', index_col=None, header=None)
    new_index_temp = ['MID'+ str(idx)]* len(data_mid);
    
    index_mid = pd.DataFrame({'Index' : new_index_temp}, index = range(0,len(data_mid)));
    data_mid = data_mid.join(index_mid)
    
    microbiome = microbiome.append(data_mid)

    
microbiome.columns = ['Name','Result', 'Index']
microbiome

In [ ]:
metadata = pd.read_excel(path + '/metadata.xls', index_col=None, header=None)
metadata.columns = ['Index','Group', 'Sample']


microbiome_final = pd.merge(microbiome, metadata,on=('Index'), how='left', indicator=False,)
microbiome_final = microbiome_final.set_index(['Index','Name']).fillna('unknown');
microbiome_final.head()

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html');

In [ ]:
titanic_data = pd.read_excel('Data/titanic.xls', sheetname='titanic', header=0)
titanic_data[['boat']] = titanic_data[['boat']].fillna(-1)
titanic_data[['body']] = titanic_data[['body']].fillna(-1)
titanic_data.head()

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

### Task 3.1
Describe the type and the value range of each attribute. Indicate and transform the attributes that can be Categorical.

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

Type: value range 

1. pclass: {1; 2; 3}
2. survived: {0; 1}
3. name: name
4. sex: {male; female}
5. age: age
6. sibsp: number of siblings/spouse aboard {}
7. parch: number of parent/children aboard
8. ticket: ticket number
9. fare: fare
10. cabin: {A,B,C,D,E,F,G,S}
11. embarked:{C,S,Q}
12. boat: escape boat number range(1,16) + A,B,C,D
13. body: corpse number
14. home.dest: home and destination

Categorisables: pclass, survived, sex, sibsp, parch, fare

### Task 3.2
Plot histograms for the travel class, embarkation port, sex and age attributes. For the latter one, use discrete decade intervals.

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
nbr_first_class = len(titanic_data[(titanic_data['pclass']==1)])
print("Number of first class passagers : " , nbr_first_class)

In [ ]:
nbr_second_class = len(titanic_data[(titanic_data['pclass']==2)])
print("Number of second class passagers : ",nbr_second_class)

In [ ]:
nbr_third_class = len(titanic_data[(titanic_data['pclass']==3)])
print("Number of third class passagers :", nbr_third_class)

In [ ]:
pclass_frame = pd.DataFrame([nbr_first_class, nbr_second_class, nbr_third_class], index= [1,2,3])
pclass_frame.columns = ['people']
pclass_frame.plot(kind='bar',title="repartition of the passagers among the classes");

In [ ]:
emb_c = len(titanic_data[(titanic_data['embarked']=='C')])
emb_q = len(titanic_data[(titanic_data['embarked']=='Q')])
emb_s = len(titanic_data[(titanic_data['embarked']=='S')])

emb_frame = pd.DataFrame([emb_c, emb_q, emb_s], index= ['Cherbourg', 'Queenstown', 'Southampton'])
emb_frame.columns = ['people']
emb_frame.plot(kind='bar',title="embarkation port");

In [ ]:
sex_m = len(titanic_data[(titanic_data['sex']=='male')])
sex_f = len(titanic_data[(titanic_data['sex']=='female')])

sex_frame = pd.DataFrame([sex_m, sex_f], index= ['male', 'female'])
sex_frame.columns = ['people']
sex_frame.plot(kind='bar',title="Sex of passagers");

In [ ]:
titanic_cpy = titanic_data.copy()
age_cut = pd.cut(titanic_data['age'], [0,10,20,30,40,50,60,70,80,90,100])
ones = pd.DataFrame([1]*len(titanic_cpy))
age_cut_ext = pd.concat([titanic_cpy['age'], ones], axis=1)
age_indexed = age_cut_ext.set_index(['age'])
grouped = age_cut_ext.groupby(pd.cut(age_cut_ext['age'], np.arange(0, 100+10, 10))).sum()
grouped = grouped.drop(['age'],axis=1)
grouped.columns = ['people']
grouped.dropna().plot(kind='bar',title="Group age of passagers");

### Task 3.3
Calculate the proportion of passengers by cabin floor. Present your results in a pie chart.

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
titanic_cabin = titanic_data['cabin'].fillna('S')
floor_A = len(titanic_cabin[titanic_cabin.str.startswith('A')])
floor_B = len(titanic_cabin[titanic_cabin.str.startswith('B')])
floor_C = len(titanic_cabin[titanic_cabin.str.startswith('C')])
floor_D = len(titanic_cabin[titanic_cabin.str.startswith('D')])
floor_E = len(titanic_cabin[titanic_cabin.str.startswith('E')])
floor_F = len(titanic_cabin[titanic_cabin.str.startswith('F')])
floor_G = len(titanic_cabin[titanic_cabin.str.startswith('G')])
floor_S = len(titanic_cabin[titanic_cabin.str.startswith('S')])
floor_T = len(titanic_cabin[titanic_cabin.str.startswith('T')])

cabin_frame = pd.DataFrame([floor_A, floor_B, floor_C, floor_D, floor_E, floor_F, 
                          floor_G, floor_S, floor_T], index = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'S', 'T'])
cabin_frame.columns = ['people']
cabin_frame.dropna().plot.pie(subplots = True,figsize = (4,4),title="Proportion of passengers per rows");

### Task 3.4
For each travel class, calculate the proportion of the passengers that survived. Present your results in pie charts.

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
first_class = titanic_data[(titanic_data['pclass']==1)]
second_class = titanic_data[(titanic_data['pclass']==2)]
third_class = titanic_data[(titanic_data['pclass']==3)]

first_class_surv = len(first_class[(first_class['survived']==1)])
second_class_surv = len(second_class[(second_class['survived']==1)])
third_class_surv = len(third_class[(third_class['survived']==1)])

first_class_dead = nbr_first_class - first_class_surv
second_class_dead = nbr_second_class - second_class_surv
third_class_dead = nbr_third_class - third_class_surv

first_df = pd.DataFrame([first_class_surv, first_class_dead], index = ['survived', 'died'])
first_df.columns = [None]
first_df.plot.pie(subplots = True, figsize = (4,4),title="First class pie chart")

second_df = pd.DataFrame([second_class_surv, second_class_dead], index = ['survied', 'died'])
second_df.columns = [None]
second_df.plot.pie(subplots = True, figsize = (4,4),title="Second class pie chart")

third_df = pd.DataFrame([third_class_surv, third_class_dead], index = ['survied', 'died'])
third_df.columns = [None]
third_df.plot.pie(subplots = True, figsize = (4,4), title ="Third class pie chart");

### Task 3.5 
Calculate the proportion of the passengers that survived by travel class and sex. Present your results in a single histogram

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
first_surv = first_class[(first_class['survived'] == 1)]
first_surv_m = len(first_surv[first_surv['sex']=='male'])
first_surv_f = len(first_surv[first_surv['sex']=='female'])

second_surv = second_class[(second_class['survived'] == 1)]
second_surv_m = len(second_surv[second_surv['sex']=='male'])
second_surv_f = len(second_surv[second_surv['sex']=='female'])

third_surv = third_class[(third_class['survived'] == 1)]
third_surv_m = len(third_surv[third_surv['sex']=='male'])
third_surv_f = len(third_surv[third_surv['sex']=='female'])

class_sex_df = pd.DataFrame([first_surv_m, first_surv_f, second_surv_m, second_surv_f, third_surv_m, third_surv_f], 
                           index = ['male_1st', 'female_1st', 'male_2nd', 'female_2nd', 'male_3rd', 'female_3rd'])
class_sex_df.columns = [None]
class_sex_df.plot.pie(subplots = True, figsize = (4,4))
class_sex_df.plot(kind='bar');

### Task 3.6
Create 2 equally populated age categories and calculate survival proportions by age category, travel class and sex. Present your results in a DataFrame with unique index.

 <div class="lev1"><a href="#Table-of-Contents"><span class="toc-item-num"></span>Table of contents</a></div>

In [ ]:
titanic_age_cpy = titanic_data.copy()
titanic_age = titanic_age_cpy.age.dropna()
pd.qcut(titanic_age, 2).head()
titanic_age_cpy[['age']] = titanic_age_cpy[['age']].dropna()
titanic_age_cpy['survived'] = titanic_age_cpy[(titanic_age_cpy['survived'] == 1)]
titanic_age_cpy = titanic_age_cpy.dropna()
total_survived = len(titanic_age_cpy)
young = titanic_age_cpy[(titanic_age_cpy['age'] <= 28)]
old = titanic_age_cpy[(titanic_age_cpy['age'] > 28)]

young_first = young[(young['pclass'] == 1)]
young_second = young[(young['pclass'] == 2)]
young_third = young[(young['pclass'] == 3)]

old_first = old[(old['pclass'] == 1)]
old_second = old[(old['pclass'] == 2)]
old_third = old[(old['pclass'] == 3)]

young_first_f = len(young_first[(young_first['sex']=='female')])*100/total_survived
young_second_f = len(young_second[(young_second['sex']=='female')])*100/total_survived
young_third_f = len(young_third[(young_third['sex']=='female')])*100/total_survived

young_first_m = len(young_first[(young_first['sex']=='male')])*100/total_survived
young_second_m = len(young_second[(young_second['sex']=='male')])*100/total_survived
young_third_m = len(young_third[(young_third['sex']=='male')])*100/total_survived

old_first_f = len(old_first[(old_first['sex']=='female')])*100/total_survived
old_second_f = len(old_second[(old_second['sex']=='female')])*100/total_survived
old_third_f = len(old_third[(old_third['sex']=='female')])*100/total_survived

old_first_m = len(old_first[(old_first['sex']=='male')])*100/total_survived
old_second_m = len(old_second[(old_second['sex']=='male')])*100/total_survived
old_third_m = len(old_third[(old_third['sex']=='male')])*100/total_survived

last_df = pd.DataFrame([young_first_f, young_first_m, young_second_f, young_second_m, young_third_f, young_third_m,
                       old_first_f, old_first_m, old_second_f, old_second_m, old_third_f, old_third_m],
                      index = ['<=28, 1st, F', '<=28, 1st, M', '<=28, 2nd, F', '<=28, 2nd, M', '<=28, 3rd, F', '<=28, 3rd, M',
                               '>28, 1st, F', '>28, 1st, M', '>28, 2nd, F', '>28, 2nd, M', '>28, 3rd, F', '>28, 3rd, M'],
                      columns = ['% survival']).round(2)
last_df